In [45]:
import funciones
import pandas as pd
import numpy as np
import pycountry as pc
pd.options.plotting.backend = "plotly"

import plotly.express as px

## CO2 por alimentos
Cargamos csv de emisiones CO2 por alimentos

In [46]:
fod = pd.read_csv('../data/Food/food-emissions-supply-chain.csv')
# fod.head()

In [47]:
fod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Entity                      43 non-null     object 
 1   Code                        0 non-null      float64
 2   Animal_origin               43 non-null     bool   
 3   Year                        43 non-null     int64  
 4   food_emissions_land_use     43 non-null     float64
 5   food_emissions_farm         43 non-null     float64
 6   food_emissions_animal_feed  43 non-null     float64
 7   food_emissions_processing   43 non-null     float64
 8   food_emissions_transport    43 non-null     float64
 9   food_emissions_retail       43 non-null     float64
 10  food_emissions_packaging    43 non-null     float64
 11  food_emissions_losses       43 non-null     float64
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 3.9+ KB


Cargamos datos de consumo de agua y añadimos al de alimentos.

In [48]:
columns = ['Entity', 'Code', 'Year', 'Water_l_kg']
wat = pd.read_csv('..\data\Food\water-withdrawals-per-kg-poore.csv', names=columns, header=0)
wat.head(1)

,Entity,Code,Year,Water_l_kg
0,Apples,NaN,2010,180.1


In [49]:
fdi = wat.set_index('Entity')['Water_l_kg'].to_dict()

fod['Water_l_kg'] = fod['Entity'].map(fdi)
fod.head()

,Entity,Code,Animal_origin,Year,food_emissions_land_use,food_emissions_farm,food_emissions_animal_feed,food_emissions_processing,food_emissions_transport,food_emissions_retail,food_emissions_packaging,food_emissions_losses,Water_l_kg
0,Apples,NaN,False,2018,-0.028951,0.225816,0.000000,0.003820,0.095804,0.016578,0.044526,0.070808,180.1
1,Bananas,NaN,False,2018,-0.025534,0.269618,0.000000,0.060146,0.295830,0.020981,0.065670,0.175185,114.5
2,Barley,NaN,False,2018,0.008676,0.176377,0.000000,0.127703,0.035351,0.263587,0.496786,0.070692,17.1
3,Beef (beef herd),NaN,True,2018,23.237536,56.228063,2.680979,1.811083,0.494125,0.233538,0.352084,14.439998,1451.2
4,Beef (dairy herd),NaN,True,2018,1.266022,21.915529,3.503673,1.547664,0.592410,0.254032,0.374549,3.847515,2714.3


Limpieza del df

In [50]:
fod.loc[fod['Entity'] == 'Shrimps (farmed)', 'Water_l_kg'] = 220
fod = fod.drop('Year', axis=1)
# fod['Entity'].value_counts()

In [51]:
fod.loc[fod['Entity'] ==  'Lamb & Mutton', 'Entity'] = 'Lamb'
fod.loc[fod['Entity'] ==  'Beef (dairy herd)', 'Entity'] = 'Beef milk'
fod.loc[fod['Entity'] ==  'Beef (beef herd)', 'Entity'] = 'Beef meat'
fod = fod.drop(index=19, axis=0).reset_index(drop=True)

Creamos una columna id para relaciones

In [54]:
fod['Code'] = [x for x in range(1, 43)]
# fod3

#### Leyenda de columnas

- Emisiones kgCO2 por kg
- Consumo de agua en litros por kg de comida

In [55]:
fod.to_csv('../data/Food/food_clean.csv', index=False)

## Soja

Cargamos datos del uso futuro de la soja recogida

In [56]:
columns = ['Entity', 'Code', 'Year', 'Food', 'Feed', 'Processed']

soy1 = pd.read_csv('..\data\Soy\soybean-production-and-use.csv', names=columns, header=1)
soy1

,Entity,Code,Year,Food,Feed,Processed
0,Afghanistan,AFG,2015,0.0,0.0,NaN
1,Afghanistan,AFG,2016,0.0,0.0,NaN
2,Afghanistan,AFG,2017,0.0,0.0,NaN
3,Afghanistan,AFG,2018,0.0,0.0,NaN
4,Afghanistan,AFG,2019,0.0,3000.0,NaN
...,...,...,...,...,...,...
12077,Zimbabwe,ZWE,2016,2000.0,NaN,54000.0
12078,Zimbabwe,ZWE,2017,4000.0,NaN,82000.0
12079,Zimbabwe,ZWE,2018,5000.0,NaN,67000.0
12080,Zimbabwe,ZWE,2019,1000.0,NaN,66000.0


In [57]:
soy1['Entity'].value_counts()

Entity
Zimbabwe                         60
Malta                            60
Low-income countries             60
Lower-middle-income countries    60
Tanzania                         60
                                 ..
Bhutan                            2
Micronesia (country)              2
Bahrain                           2
Nauru                             2
South Sudan                       2
Name: count, Length: 239, dtype: int64

Cargamos datos de la producción y consumo de soja

In [58]:
columns = ['Entity', 'Code', 'Year', 'Production', 'Yield', 'Area harvested']

soy2 = pd.read_csv('..\data\Soy\soy-production-yield-area.csv', names=columns, header=1)
soy2

,Entity,Code,Year,Production,Yield,Area harvested
0,Africa,NaN,1962,84594.0,0.393471,214994.0
1,Africa,NaN,1963,87260.0,0.394851,220995.0
2,Africa,NaN,1964,76781.0,0.369237,207945.0
3,Africa,NaN,1965,73067.0,0.347998,209964.0
4,Africa,NaN,1966,77180.0,0.374569,206050.0
...,...,...,...,...,...,...
7421,Zimbabwe,ZWE,2017,36478.0,1.551300,23515.0
7422,Zimbabwe,ZWE,2018,69688.0,1.868000,37307.0
7423,Zimbabwe,ZWE,2019,23043.0,0.695400,33136.0
7424,Zimbabwe,ZWE,2020,59656.0,1.814700,32874.0


Unimos los df en uno conjunto y lo limpiamos

In [59]:
soy2 = soy2.drop(['Code'], axis=1)
soy = pd.merge(soy1, soy2, on=['Entity', 'Year']).reset_index(drop=True)
soy.shape

(7104, 9)

In [60]:
soy = soy[~soy['Entity'].str.contains('(FAO)')].reset_index(drop=True)
soy['Entity'].value_counts()


C:\Users\Nerea\AppData\Local\Temp\ipykernel_11876\839986000.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  soy = soy[~soy['Entity'].str.contains('(FAO)')].reset_index(drop=True)


Entity
Liberia        60
Mexico         60
Pakistan       60
Oceania        60
North Korea    60
               ..
Malta           3
Bhutan          2
Cyprus          2
Luxembourg      1
Finland         1
Name: count, Length: 131, dtype: int64

In [61]:
# lst = soy['Entity'].value_counts().index.tolist()
# lst

In [62]:
soy.loc[soy['Entity'] == 'European Union (27)', 'Entity'] = 'European Union'
soy.loc[soy['Entity'] =="Cote d'Ivoire", 'Entity'] = 'Ivory Coast'
soy.loc[soy['Entity'] == 'Democratic Republic of Congo', 'Entity'] = 'Congo'

In [63]:
soy.head(10)

,Entity,Code,Year,Food,Feed,Processed,Production,Yield,Area harvested
0,Africa,NaN,1962,44000.0,7000.0,17000.0,84594.0,0.393471,214994.0
1,Africa,NaN,1963,31000.0,7000.0,5000.0,87260.0,0.394851,220995.0
2,Africa,NaN,1964,43000.0,6000.0,14000.0,76781.0,0.369237,207945.0
3,Africa,NaN,1965,34000.0,6000.0,12000.0,73067.0,0.347998,209964.0
4,Africa,NaN,1966,41000.0,6000.0,2000.0,77180.0,0.374569,206050.0
5,Africa,NaN,1967,48000.0,6000.0,4000.0,80884.0,0.418871,193100.0
6,Africa,NaN,1968,49000.0,7000.0,3000.0,90459.0,0.416296,217295.0
7,Africa,NaN,1969,52000.0,6000.0,6000.0,89000.0,0.459376,193741.0
8,Africa,NaN,1970,53000.0,6000.0,8000.0,97211.0,0.436648,222630.0
9,Africa,NaN,1971,60000.0,6000.0,2000.0,97545.0,0.412102,236701.0


In [64]:
soy['Year'].isnull().any()

False

In [65]:
soy['Year'] = pd.to_datetime(soy['Year'], format='%Y')

In [66]:
columns_to_convert = ['Food', 'Feed', 'Processed', 'Production', 'Yield', 'Area harvested']

for column in columns_to_convert:
    soy[column] = pd.to_numeric(soy[column], errors='coerce')

In [67]:
soy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5469 entries, 0 to 5468
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Entity          5469 non-null   object        
 1   Code            4810 non-null   object        
 2   Year            5469 non-null   datetime64[ns]
 3   Food            5369 non-null   float64       
 4   Feed            3090 non-null   float64       
 5   Processed       4404 non-null   float64       
 6   Production      5467 non-null   float64       
 7   Yield           5094 non-null   float64       
 8   Area harvested  5463 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 384.7+ KB


In [68]:
soy['Code'] = pd.factorize(soy['Entity'])[0]+1
soy.head()

,Entity,Code,Year,Food,Feed,Processed,Production,Yield,Area harvested
0,Africa,1,1962-01-01,44000.0,7000.0,17000.0,84594.0,0.393471,214994.0
1,Africa,1,1963-01-01,31000.0,7000.0,5000.0,87260.0,0.394851,220995.0
2,Africa,1,1964-01-01,43000.0,6000.0,14000.0,76781.0,0.369237,207945.0
3,Africa,1,1965-01-01,34000.0,6000.0,12000.0,73067.0,0.347998,209964.0
4,Africa,1,1966-01-01,41000.0,6000.0,2000.0,77180.0,0.374569,206050.0


In [69]:
len(soy['Entity'].value_counts()) == len(soy['Code'].value_counts())

True

In [70]:
soy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5469 entries, 0 to 5468
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Entity          5469 non-null   object        
 1   Code            5469 non-null   int64         
 2   Year            5469 non-null   datetime64[ns]
 3   Food            5369 non-null   float64       
 4   Feed            3090 non-null   float64       
 5   Processed       4404 non-null   float64       
 6   Production      5467 non-null   float64       
 7   Yield           5094 non-null   float64       
 8   Area harvested  5463 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 384.7+ KB


#### Leyenda de las columnas

- Soja usada para comida humana, soja usada para alimentación de ganado y soja usada para procesados o aceites. Todo en toneladas.
-  Soja producida en t, soja utilizada en comercio de t/ha, superficie de area plantada ha

In [71]:
soy.to_csv('../data/Soy/soy_clean.csv', index=False)

## Aceites 

Cargamos datos del consumo y producción de los aceites según su origen

In [72]:
columns = ['Entity', 'Code', 'Year', 'Coconut', 'Groundnut', 'Palm', 'Sesame', 'Sunflower', 'Cottonseed', 'Olive', 'Rapeseed', 'Soybean']

oil_yield = pd.read_csv('..\data\Palm\oil-yield-by-crop.csv', names=columns, header=1)
oil_yield

,Entity,Code,Year,Coconut,Groundnut,Palm,Sesame,Sunflower,Cottonseed,Olive,Rapeseed,Soybean
0,Afghanistan,AFG,1962,NaN,NaN,NaN,0.056506,0.378042,0.084737,0.1496,NaN,NaN
1,Afghanistan,AFG,1963,NaN,NaN,NaN,0.055145,0.378042,0.096713,0.1360,NaN,NaN
2,Afghanistan,AFG,1964,NaN,NaN,NaN,0.082000,0.378042,0.065567,0.1496,NaN,NaN
3,Afghanistan,AFG,1965,NaN,NaN,NaN,0.084964,0.390090,0.097932,0.1360,NaN,NaN
4,Afghanistan,AFG,1966,NaN,NaN,NaN,0.094843,0.414187,0.134445,0.1496,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12162,Zimbabwe,ZWE,2016,NaN,0.037665,NaN,NaN,0.166113,0.061971,NaN,NaN,0.240391
12163,Zimbabwe,ZWE,2017,NaN,0.036669,NaN,NaN,0.181400,0.082358,NaN,NaN,0.612375
12164,Zimbabwe,ZWE,2018,NaN,0.049197,NaN,NaN,0.170973,0.074162,NaN,NaN,0.394028
12165,Zimbabwe,ZWE,2019,NaN,0.060487,NaN,NaN,0.097837,0.110400,NaN,NaN,0.250483


In [73]:
columns = ['Entity', 'Code', 'Year', 'Sesame', 'Maize', 'Safflower', 'Coconut', 'Sunflower', 'Rapeseed', 'Palm kernel', 'Groundnut', 'Linseed', 'Olive', 'Palm', 'Soybean', 'Cottonseed']

oil_prod = pd.read_csv(r"..\data\Palm\vegetable-oil-production.csv", names=columns, header=1)
oil_prod

,Entity,Code,Year,Sesame,Maize,Safflower,Coconut,Sunflower,Rapeseed,Palm kernel,Groundnut,Linseed,Olive,Palm,Soybean,Cottonseed
0,Afghanistan,AFG,2011,1292.0,NaN,NaN,NaN,60.0,NaN,NaN,NaN,674.65,538.56,NaN,NaN,3725.00
1,Afghanistan,AFG,2012,1261.0,NaN,NaN,NaN,61.0,NaN,NaN,NaN,706.30,538.56,NaN,NaN,4097.97
2,Afghanistan,AFG,2013,1334.0,NaN,NaN,NaN,138.0,NaN,NaN,NaN,733.57,545.93,NaN,NaN,4761.86
3,Afghanistan,AFG,2014,1250.0,NaN,NaN,NaN,960.0,NaN,NaN,NaN,790.00,658.42,NaN,NaN,4832.28
4,Afghanistan,AFG,2015,861.0,NaN,NaN,NaN,1035.0,NaN,NaN,NaN,5374.63,711.37,NaN,NaN,5690.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12703,Zimbabwe,ZWE,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7560.0,NaN,NaN,NaN,NaN,540.00
12704,Zimbabwe,ZWE,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6300.0,NaN,NaN,NaN,NaN,1260.00
12705,Zimbabwe,ZWE,1966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4956.0,NaN,NaN,NaN,NaN,1260.00
12706,Zimbabwe,ZWE,1967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7518.0,NaN,NaN,NaN,NaN,3474.00


In [74]:
oil_yield.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12167 entries, 0 to 12166
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Entity      12167 non-null  object 
 1   Code        9589 non-null   object 
 2   Year        12167 non-null  int64  
 3   Coconut     5396 non-null   float64
 4   Groundnut   6774 non-null   float64
 5   Palm        3584 non-null   float64
 6   Sesame      4345 non-null   float64
 7   Sunflower   5470 non-null   float64
 8   Cottonseed  7127 non-null   float64
 9   Olive       3369 non-null   float64
 10  Rapeseed    4762 non-null   float64
 11  Soybean     5594 non-null   float64
dtypes: float64(9), int64(1), object(2)
memory usage: 1.1+ MB


Limpieza de los df. Decidimos dejarlos separados para un mejor estudio de ellos a posteriori pero creamos una columna con id para las relaciones.

In [75]:
oil_yield = oil_yield[~oil_yield['Entity'].str.contains('(FAO)')].reset_index(drop=True)
oil_yield.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10300 entries, 0 to 10299
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Entity      10300 non-null  object 
 1   Code        9589 non-null   object 
 2   Year        10300 non-null  int64  
 3   Coconut     4201 non-null   float64
 4   Groundnut   5139 non-null   float64
 5   Palm        2590 non-null   float64
 6   Sesame      3027 non-null   float64
 7   Sunflower   3949 non-null   float64
 8   Cottonseed  5478 non-null   float64
 9   Olive       2288 non-null   float64
 10  Rapeseed    3342 non-null   float64
 11  Soybean     4026 non-null   float64
dtypes: float64(9), int64(1), object(2)
memory usage: 965.8+ KB


C:\Users\Nerea\AppData\Local\Temp\ipykernel_11876\854352433.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  oil_yield = oil_yield[~oil_yield['Entity'].str.contains('(FAO)')].reset_index(drop=True)


In [76]:
oil_yield.loc[oil_yield['Entity'] == 'European Union (27)', 'Entity'] = 'European Union'
oil_yield.loc[oil_yield['Entity'] =="Cote d'Ivoire", 'Entity'] = 'Ivory Coast'
oil_yield.loc[oil_yield['Entity'] == 'Democratic Republic of Congo'] = 'Congo'
oil_yield.loc[oil_yield['Entity'] == 'Micronesia (country)'] = 'Micronesia'

In [77]:
oil_prod = oil_prod[~oil_prod['Entity'].str.contains('(FAO)')].reset_index(drop=True)
oil_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Entity       10841 non-null  object 
 1   Code         10130 non-null  object 
 2   Year         10841 non-null  int64  
 3   Sesame       739 non-null    float64
 4   Maize        3419 non-null   float64
 5   Safflower    320 non-null    float64
 6   Coconut      5869 non-null   float64
 7   Sunflower    988 non-null    float64
 8   Rapeseed     819 non-null    float64
 9   Palm kernel  3785 non-null   float64
 10  Groundnut    6065 non-null   float64
 11  Linseed      4061 non-null   float64
 12  Olive        2485 non-null   float64
 13  Palm         3153 non-null   float64
 14  Soybean      1208 non-null   float64
 15  Cottonseed   6166 non-null   float64
dtypes: float64(13), int64(1), object(2)
memory usage: 1.3+ MB


C:\Users\Nerea\AppData\Local\Temp\ipykernel_11876\2797232385.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  oil_prod = oil_prod[~oil_prod['Entity'].str.contains('(FAO)')].reset_index(drop=True)


In [78]:
oil_prod.loc[oil_prod['Entity'] == 'European Union (27)', 'Entity'] = 'European Union'
oil_prod.loc[oil_prod['Entity'] =="Cote d'Ivoire", 'Entity'] = 'Ivory Coast'
oil_prod.loc[oil_prod['Entity'] == 'Democratic Republic of Congo'] = 'Congo'
oil_prod.loc[oil_prod['Entity'] == 'Micronesia (country)'] = 'Micronesia'


In [79]:
# Dejamos los valores nulos porque representan que no existe la producción de ese vegetale en ese momento. Rellenarlos con valores sería sesgar la info.

In [80]:
# Creamos un diccionario con los nombres de paises y su id. Se aplica mapeando a los dos df de oil.

dic = soy.set_index('Entity')['Code'].to_dict()
oil_prod['Code'] = oil_prod['Entity'].map(dic)
oil_yield['Code'] = oil_yield['Entity'].map(dic)
max(soy['Code'])

131

In [81]:
nul_ind = oil_prod['Code'].isnull()
ent = oil_prod.loc[nul_ind, 'Entity'].unique()
fac = pd.factorize(ent)[0] + 132
ma = dict(zip(ent, fac))
ser = oil_prod['Entity'].map(ma)

oil_prod.loc[nul_ind, 'Code'] = ser
oil_prod['Code'] = oil_prod['Code'].astype(int)
# oil_prod['Code'].value_counts()


In [82]:
nul_ind = oil_yield['Code'].isnull()
ent = oil_yield.loc[nul_ind, 'Entity'].unique()
fac = pd.factorize(ent)[0] + 132
ma = dict(zip(ent, fac))
ser = oil_yield['Entity'].map(ma)

oil_yield.loc[nul_ind, 'Code'] = ser
oil_yield['Code'] = oil_yield['Code'].astype(int)
# oil_prod['Code'].value_counts()

In [83]:
oil_prod.to_csv('../data/Palm/oil_prod_clean.csv', index=False)
oil_yield.to_csv('../data/Palm/oil_yield_clean.csv', index=False)

#### Habitantes por regiones

Creamos el df de habitantes y le asignamos el id.

In [84]:
pop = pd.read_csv('../data/Population/population.csv')
pop.head()

,Entity,Population
0,Congo,89561403
1,Malawi,19930763
2,North Korea,25778816
3,Nepal,29996478
4,Netherlands,17134873


In [85]:
ids = oil_yield.set_index('Entity')['Code'].to_dict()
pop['Code'] = pop['Entity'].map(ids)
# hab['Code'] = hab['Code'].astype(int)
pop.loc[pop['Code'].isnull(), 'Entity']

Series([], Name: Entity, dtype: object)

In [86]:
print(set(pop['Entity']) - set(oil_yield['Entity']))

set()


In [87]:
pop.to_csv('../data/Population/population_clean.csv')

#### Emisiones por 100g de proteina

Cargamos df y lo indeamos.

In [89]:
columns = ['Entity', 'Code', 'Year', 'Emissions_100g']

prot = pd.read_csv('..\data\Protein\protein.csv', names=columns, header=1)
prot.head()

,Entity,Code,Year,Emissions_100g
0,Bananas,NaN,2010,9.555556
1,Beef (beef herd),NaN,2010,49.889669
2,Beef (dairy herd),NaN,2010,16.869301
3,Berries & Grapes,NaN,2010,15.300000
4,Brassicas,NaN,2010,4.636364


In [90]:
prot = prot.drop('Year', axis=1)
prot.loc[prot['Entity'] ==  'Lamb & Mutton', 'Entity'] = 'Lamb meat'
prot.loc[prot['Entity'] ==  'Beef (dairy herd)', 'Entity'] = 'Beef milk'
prot.loc[prot['Entity'] ==  'Beef (beef herd)', 'Entity'] = 'Beef meat'
prot = prot.drop(index=16, axis=0).reset_index(drop=True)



In [91]:
pdi = fod.set_index('Entity')['Code'].to_dict()
prot['Code'] = prot['Entity'].map(pdi)

nul_ind = prot['Code'].isnull()
ent = prot.loc[nul_ind, 'Entity'].unique()
fac = pd.factorize(ent)[0] + 43
ma = dict(zip(ent, fac))
ser = prot['Entity'].map(ma)

prot.loc[nul_ind, 'Code'] = ser
prot['Code'] = prot['Code'].astype(int)
prot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Entity          30 non-null     object 
 1   Code            30 non-null     int32  
 2   Emissions_100g  30 non-null     float64
dtypes: float64(1), int32(1), object(1)
memory usage: 732.0+ bytes


In [93]:
prot.to_csv('../data/Protein/protein_clean.csv',index=False)